In [1]:
%matplotlib
%matplotlib notebook

# Import Libraries
import cv2
import numpy as np
import numpy.matlib
import pandas as pd
import os
import struct
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm_notebook
import pickle

# Headers for different tables
meta_data_header = ['FrameNumber', 'Time', 'Stage_x', 'Stage_y', 'Centroid_x', 'Centroid_y',
                    'Midpoint_x', 'Midpoint_y', 'Head_x', 'Head_y', 'Tail_x', 'Tail_y', 'MouthHook_x', 'MouthHook_y',
                    'LeftMHhook_x', 'LeftMHhook_y', 'RightMHhook_x', 'RightMHhook_y',
                    'LeftDorsalOrgan_x', 'LeftDorsalOrgan_y', 'RightDorsalOrgan_x', 'RightDorsalOrgan_y',
                    'CenterBolwigOrgan_x', 'CenterBolwigOrgan_y', 'LeftBolwigOrgan_x', 'LeftBolwigOrgan_y',
                    'RightBolwigOrgan_x', 'RightBolwigOrgan_y', 'V9_x', 'V9_y', 'V10_x', 'V10_y', 'V11_x', 'V11_y',
                    'V12_x', 'V12_y', 'V13_x', 'V13_y', 'V14_x', 'V14_y', 'V15_x', 'V15_y', 'V16_x', 'V16_y',
                    'MouthHook_votes', 'LeftMHhook_votes', 'RightMHhook_votes', 'LeftDorsalOrgan_votes',
                    'RightDorsalOrgan_votes', 'CenterBolwigOrgan_votes', 'LeftBolwigOrgan_votes', 'RightBolwigOrgan_votes',
                    'V9_votes', 'V10_votes', 'V11_votes', 'V12_votes', 'V13_votes', 'V14_votes', 'V15_votes',
                    'V16_votes', 'Num_Key_points']

coordinate_header = ['FrameNumber', 'MouthHook_x', 'MouthHook_y', 'LeftMHhook_x', 'LeftMHhook_y',
                     'RightMHhook_x', 'RightMHhook_y', 'LeftDorsalOrgan_x', 'LeftDorsalOrgan_y',
                     'RightDorsalOrgan_x', 'RightDorsalOrgan_y', 'CenterBolwigOrgan_x', 'CenterBolwigOrgan_y',
                     'LeftBolwigOrgan_x', 'LeftBolwigOrgan_y', 'RightBolwigOrgan_x', 'RightBolwigOrgan_y']

distance_header = ['MouthHook', 'LeftMHhook',
                   'RightMHhook', 'LeftDorsalOrgan', 'RightDorsalOrgan',
                   'CenterBolwigOrgan', 'LeftBolwigOrgan', 'RightBolwigOrgan']

roc_headers = ['true_positive', 'false_positive', 'true_negative',
               'false_negative', 'positive', 'negative',
               'true_positive_rate', 'false_positive_rate', 'total_negative',
               'total_positive', 'true_negative_rate', 'false_negative_rate',
               'precision', 'recall', 'vote_threshold']


def getROCcurve(allData, detectBodyParts, outlierDistAll, vote_thresholds):
    rocDataAll = {}
    for bp in tqdm_notebook(detectBodyParts, total=len(detectBodyParts), unit="bp", desc='Body parts'):
        rocDataAll[bp] = {}
        for out_dist in tqdm_notebook(outlierDistAll,  total=len(outlierDistAll), unit="thresh", desc='Outliers', leave=False):
            rocDataAll[bp][out_dist] = pd.DataFrame([], columns=roc_headers)
            vt_iter = -1
            for vt in tqdm_notebook(vote_thresholds, total=len(vote_thresholds), unit="vote", desc='Votes', leave=False):
                vt_iter += 1
                true_positive = 0
                false_positive = 0
                true_negative = 0
                false_negative = 0
                positives = 0
                negatives = 0

                # For each Vote Threshold Find TP, FP etc.
                for row_index, row in allData.iterrows():
                    annotation = row.loc[[bp+'_x_A', bp+'_y_A']].values
                    detection = row.loc[[bp+'_x_T', bp+'_y_T']].values
                    total_kp = row.loc['Num_Key_points']
                    vote = float(row.loc[[bp+'_votes']].values)
                    distance = row.loc[[bp+'_dist']].values
                    detection_present = False
                    annotation_present = False

                    if (annotation[0] >= 0) and (annotation[1] >= 0):
                        positives += 1
                        annotation_present = True
                        if (vote > vt) and (distance <= out_dist):
                            detection_present = True
                    else:
                        negatives += 1
                        if (vote > vt):
                            detection_present = True

                    if annotation_present and detection_present:
                        true_positive += 1
                    elif annotation_present and (not detection_present):
                        false_negative += 1
                    elif (not annotation_present) and (not detection_present):
                        true_negative += 1
                    elif (not annotation_present) and detection_present:
                        false_positive += 1

            #             print 'P(%d) = TP(%d)+FN(%d)' %(positives, true_positive, false_negative)
            #             print 'N(%d) = TN(%d)+FP(%d)' %(negatives, true_negative, false_positive)

                assert positives == (true_positive+false_negative), sprintf(
                    'positives(%d) ~= true_positive(%d)+false_negative(%d)+false_positive(%d)', positives, true_positive, false_negative, false_positive)
                assert negatives == (true_negative+false_positive), sprintf(
                    'negatives(%d) == true_negative(%d)+false_positive(%d)', negatives, true_negative, false_positive)

                rocDataAll[bp][out_dist].loc[vt_iter, 'vote_threshold'] = vt
                rocDataAll[bp][out_dist].loc[vt_iter,
                                             'true_positive'] = true_positive
                rocDataAll[bp][out_dist].loc[vt_iter,
                                             'false_positive'] = false_positive
                rocDataAll[bp][out_dist].loc[vt_iter,
                                             'true_negative'] = true_negative
                rocDataAll[bp][out_dist].loc[vt_iter,
                                             'false_negative'] = false_negative
                rocDataAll[bp][out_dist].loc[vt_iter, 'positive'] = positives
                rocDataAll[bp][out_dist].loc[vt_iter,
                                             'total_negative'] = false_positive + true_negative
                rocDataAll[bp][out_dist].loc[vt_iter,
                                             'total_positive'] = true_positive + false_negative
                rocDataAll[bp][out_dist].loc[vt_iter, 'negative'] = negatives
                rocDataAll[bp][out_dist].loc[vt_iter, 'true_positive_rate'] = float(
                    true_positive)/float(positives)

                if negatives > 0:
                    rocDataAll[bp][out_dist].loc[vt_iter, 'false_positive_rate'] = float(
                        false_positive)/float(negatives)
                    rocDataAll[bp][out_dist].loc[vt_iter, 'true_negative_rate'] = float(
                        true_negative)/float(negatives)
                else:
                    rocDataAll[bp][out_dist].loc[vt_iter,
                                                 'false_positive_rate'] = 0.0
                    rocDataAll[bp][out_dist].loc[vt_iter,
                                                 'true_negative_rate'] = 0.0

                rocDataAll[bp][out_dist].loc[vt_iter, 'false_negative_rate'] = float(
                    false_negative)/float(positives)

                if (float(true_positive) + float(false_positive)) > 0.0:
                    rocDataAll[bp][out_dist].loc[vt_iter, 'precision'] = float(
                        true_positive)/(float(true_positive) + float(false_positive))
                else:
                    rocDataAll[bp][out_dist].loc[vt_iter, 'precision'] = 1

                if (float(true_positive) + float(false_negative)) > 0.0:
                    rocDataAll[bp][out_dist].loc[vt_iter, 'recall'] = float(
                        true_positive)/(float(true_positive) + float(false_negative))
                else:
                    rocDataAll[bp][out_dist].loc[vt_iter, 'recall'] = np.nan

    return rocDataAll


def getROCcurve_Combi(allData, detectBodyParts, outlierDistAll, vote_thresholds):
    rocDataAll = {}
    for bp in tqdm_notebook(detectBodyParts, total=len(detectBodyParts), unit="bp", desc='Body parts'):
        rocDataAll[bp] = {}
        for out_dist in tqdm_notebook(outlierDistAll,  total=len(outlierDistAll), unit="thresh", desc='Outliers', leave=False):
            rocDataAll[bp][out_dist] = pd.DataFrame([], columns=roc_headers)
            vt_iter = -1
            for vt in tqdm_notebook(vote_thresholds, total=len(vote_thresholds), unit="vote", desc='Votes', leave=False):
                vt_iter += 1
                true_positive = 0
                false_positive = 0
                true_negative = 0
                false_negative = 0
                positives = 0
                negatives = 0

                # For each Vote Threshold Find TP, FP etc.
                annotation = {}
                detection = {}
                distance = {}
                vote = {}
                for row_index, row in allData.iterrows():
                    annotation['ldo'] = row.loc[[
                        'LeftDorsalOrgan_x_A', 'LeftDorsalOrgan_y_A']].values
                    annotation['rdo'] = row.loc[[
                        'RightDorsalOrgan_x_A', 'RightDorsalOrgan_y_A']].values
                    detection['ldo'] = row.loc[[
                        'LeftDorsalOrgan_x_T', 'LeftDorsalOrgan_y_T']].values
                    detection['rdo'] = row.loc[[
                        'RightDorsalOrgan_x_T', 'RightDorsalOrgan_y_T']].values
                    vote['ldo'] = row.loc[['LeftDorsalOrgan_votes']].values
                    vote['rdo'] = row.loc[['RightDorsalOrgan_votes']].values
                    distance['ldo'] = row.loc[['LeftDorsalOrgan_dist']].values
                    distance['rdo'] = row.loc[['RightDorsalOrgan_dist']].values
                    total_kp = row.loc['Num_Key_points']
                    annotation_present = False
                    detection_present = False
                    
                    if bp == 'And':
                        if ((annotation['ldo'][0] >= 0) and (annotation['ldo'][1] >= 0)) and ((annotation['rdo'][0] >= 0) and (annotation['rdo'][1] >= 0)):
                            positives += 1
                            annotation_present = True
                            if ((vote['rdo'] > vt) and (distance['ldo'] <= out_dist)) and ((vote['rdo'] > vt) and (distance['ldo'] <= out_dist)):
                                detection_present = True
                        else:
                            negatives += 1
                            if (vote['ldo'] > vt) and (vote['rdo'] > vt):
                                detection_present = True
                            
                    elif bp == 'Or':
                        if ((annotation['ldo'][0] >= 0) and (annotation['ldo'][1] >= 0)) or ((annotation['rdo'][0] >= 0) and (annotation['rdo'][1] >= 0)):
                            positives += 1
                            annotation_present = True
                            if ((vote['rdo'] > vt) and (distance['ldo'] <= out_dist)) or ((vote['rdo'] > vt) and (distance['ldo'] <= out_dist)):
                                detection_present = True
                        else:
                            negatives += 1
                            if (vote['ldo'] > vt) or (vote['rdo'] > vt):
                                detection_present = True
                    
                    if annotation_present and detection_present:
                        true_positive += 1
                    elif annotation_present and (not detection_present):
                        false_negative += 1
                    elif (not annotation_present) and (not detection_present):
                        true_negative += 1
                    elif (not annotation_present) and detection_present:
                        false_positive += 1

#                 print "Condition : ", bp
#                 print 'P(%d) = TP(%d)+FN(%d)' % (positives,
#                                                  true_positive, false_negative)
#                 print 'N(%d) = TN(%d)+FP(%d)' % (negatives,
#                                                  true_negative, false_positive)

                assert positives == (true_positive+false_negative), sprintf('positives(%d) ~= true_positive(%d)+false_negative(%d)+false_positive(%d)', positives, true_positive, false_negative, false_positive)
                assert negatives == (true_negative+false_positive), sprintf('negatives(%d) == true_negative(%d)+false_positive(%d)', negatives, true_negative, false_positive)

                rocDataAll[bp][out_dist].loc[vt_iter, 'vote_threshold'] = vt
                rocDataAll[bp][out_dist].loc[vt_iter,
                                             'true_positive'] = true_positive
                rocDataAll[bp][out_dist].loc[vt_iter,
                                             'false_positive'] = false_positive
                rocDataAll[bp][out_dist].loc[vt_iter,
                                             'true_negative'] = true_negative
                rocDataAll[bp][out_dist].loc[vt_iter,
                                             'false_negative'] = false_negative
                rocDataAll[bp][out_dist].loc[vt_iter,
                                             'positive'] = positives
                rocDataAll[bp][out_dist].loc[vt_iter,
                                             'total_negative'] = false_positive + true_negative
                rocDataAll[bp][out_dist].loc[vt_iter,
                                             'total_positive'] = true_positive + false_negative
                rocDataAll[bp][out_dist].loc[vt_iter,
                                             'negative'] = negatives
                rocDataAll[bp][out_dist].loc[vt_iter, 'true_positive_rate'] = float(
                    true_positive)/float(positives)

                if negatives > 0:
                    rocDataAll[bp][out_dist].loc[vt_iter, 'false_positive_rate'] = float(
                        false_positive)/float(negatives)
                    rocDataAll[bp][out_dist].loc[vt_iter, 'true_negative_rate'] = float(
                        true_negative)/float(negatives)
                else:
                    rocDataAll[bp][out_dist].loc[vt_iter,
                                                 'false_positive_rate'] = 0.0
                    rocDataAll[bp][out_dist].loc[vt_iter,
                                                 'true_negative_rate'] = 0.0

                rocDataAll[bp][out_dist].loc[vt_iter, 'false_negative_rate'] = float(
                    false_negative)/float(positives)

                if (float(true_positive) + float(false_positive)) > 0.0:
                    rocDataAll[bp][out_dist].loc[vt_iter, 'precision'] = float(
                        true_positive)/(float(true_positive) + float(false_positive))
                else:
                    rocDataAll[bp][out_dist].loc[vt_iter, 'precision'] = 1

                if (float(true_positive) + float(false_negative)) > 0.0:
                    rocDataAll[bp][out_dist].loc[vt_iter, 'recall'] = float(
                        true_positive)/(float(true_positive) + float(false_negative))
                else:
                    rocDataAll[bp][out_dist].loc[vt_iter, 'recall'] = np.nan

    return rocDataAll


def plotRecall_curve(rocDat, detectBodyParts, outlierDistAll, figName):
    col = 0
    fig, ax = plt.subplots(1, 2, figsize=(10, 6), dpi=90)
    for bp in detectBodyParts:
        for out_thresh in outlierDistAll:
            recall = np.array(
                rocDat[bp][out_thresh].loc[:, 'recall'].values, dtype=np.float16)
            vote_threshold = np.array(
                rocDat[bp][out_thresh].loc[:, 'vote_threshold'].values, dtype=np.float16)
            ax[col].plot(vote_threshold, recall,
                         linewidth=3, label=int(out_thresh))

        ax[col].set_xlabel('Vote Threshold', fontsize=18)
        ax[col].set_ylabel('True Positive Rate', fontsize=18)
        ax[col].set_title('%s' % (bp), fontsize=20)
#         ax[col].set_aspect('equal', adjustable='box')
        ax[col].set_xlim([-0.05, max(vote_threshold) + 0.05])
        ax[col].set_ylim([-0.05, 1.05])
        ax[col].spines['right'].set_visible(False)
        ax[col].spines['top'].set_visible(False)
        ax[col].legend(loc='upper right', frameon=False,
                       fontsize=14, title='Distance')
        for label in (ax[col].get_xticklabels() + ax[col].get_yticklabels()):
            label.set_fontsize(18)
        col += 1

    plt.savefig(figName, dpi=300)


def plotROC_curve(rocDat, detectBodyParts, outlierDistAll, figName):
    col = 0
    fig, ax = plt.subplots(1, 2, figsize=(10, 6), dpi=90)

    for bp in detectBodyParts:
        for out_thresh in outlierDistAll:
            print col
            fpr = np.array(
                rocDat[bp][out_thresh].loc[:, 'false_positive_rate'].values, dtype=np.float16)
            tpr = np.array(
                rocDat[bp][out_thresh].loc[:, 'true_positive_rate'].values, dtype=np.float16)
            ax[col].plot(fpr, tpr, linewidth=3, label=int(out_thresh))

        ax[col].set_xlabel('False Positive Rate', fontsize=18)
        ax[col].set_ylabel('True Positive Rate', fontsize=18)
        ax[col].set_title(bp, fontsize=20)
        ax[col].set_aspect('equal', adjustable='box')
        ax[col].set_xlim([-0.05, 1.05])
        ax[col].set_ylim([-0.05, 1.05])
        ax[col].spines['right'].set_visible(False)
        ax[col].spines['top'].set_visible(False)
        ax[col].legend(loc='lower right', frameon=False,
                       fontsize=14, title='Distance')
        for label in (ax[col].get_xticklabels() + ax[col].get_yticklabels()):
            label.set_fontsize(18)

        col += 1

    plt.savefig(figName, dpi=300)


def plotPR_curve(rocDat, detectBodyParts, outlierDistAll, figName):
    col = 0
    fig, ax = plt.subplots(1, 2, figsize=(10, 6), dpi=90)

    for bp in detectBodyParts:
        for out_thresh in outlierDistAll:
            precision = np.array(
                rocDat[bp][out_thresh].loc[:, 'precision'].values, dtype=np.float16)
            recall = np.array(
                rocDat[bp][out_thresh].loc[:, 'recall'].values, dtype=np.float16)
            average_precision = np.sum(np.multiply(
                (recall[1:]-recall[0:-1]), precision[1:]))

            ax[col].plot(recall, precision, linewidth=3, label=int(out_thresh))

        ax[col].set_xlabel('Recall', fontsize=18)
        ax[col].set_ylabel('Precision', fontsize=18)
        ax[col].set_title('%s' % (bp), fontsize=20)
        ax[col].set_aspect('equal', adjustable='box')
        ax[col].set_xlim([-0.05, 1.05])
        ax[col].set_ylim([-0.05, 1.05])
        ax[col].spines['right'].set_visible(False)
        ax[col].spines['top'].set_visible(False)
        ax[col].legend(loc='lower left', frameon=False,
                       fontsize=14, title='Distance')
        for label in (ax[col].get_xticklabels() + ax[col].get_yticklabels()):
            label.set_fontsize(18)

        col += 1

    plt.savefig(figName, dpi=300)
    

Using matplotlib backend: MacOSX


In [2]:
root = '../expts/'
# all_exp = ['20180417_Individual', '20180417_7c1_Hess_100_LeftDO',
#            '20180417_7c1_Hess_100_BothDO', '20180417_7c1_Hess_100',
#            '20180417_7c1_Hess_50_LeftDO', '20180417_7c1_Hess_50_BothDO',
#            '20180417_7c1_Hess_50', '20180417_7c1_Hess_25',
#            '20180417_7c1_Nbr_25', '20170317_7c0', '20170318_5c0_test_180417']
all_exp = ['20180417_7c1_Hess_100_BothDO', '20180417_7c1_Hess_50_BothDO']

all_exp_names = ['Same',
                 'Hess_100_LeftDO', 'Hess_100_BothDO', 'Hess_100',
                 'Hess_50_LeftDO', 'Hess_50_BothDO', 'Hess_50',
                 'Hess_25', 'Hess_100_Nbr_25',
                 '2017_T1', '2017_T2']

test_list = [('dataCollectedOn_20180417_grp_1/Rawdata_20180417_084502_20180423_075225/', "Rawdata_20180417_084502"),
             ('dataCollectedOn_20180417_grp_2/Rawdata_20180417_083618_20180420_104633/', "Rawdata_20180417_083618"),
             ('dataCollectedOn_20180417_grp_3/Rawdata_20180417_082627_20180525_112343_A_133410/', "Rawdata_20180417_082627"),
             ('dataCollectedOn_20180417_grp_4/Rawdata_20180417_075246_20180420_092232_A_095851/', "Rawdata_20180417_075246"),
             ('dataCollectedOn_20180417_grp_5/Rawdata_20180417_072307_20180523_100127_A_122116/', "Rawdata_20180417_072307"),
             ('dataCollectedOn_20180417_grp_6/Rawdata_20180417_070739_20180522_151012/', "Rawdata_20180417_070739"),
             ('dataCollectedOn_20180417_grp_7/Rawdata_20180417_065725_20180522_114055/', "Rawdata_20180417_065725")]

vote_thresholds = np.arange(0, 65, 2)
outlierDistAll = [50, 100, 200]

run_basic = False
if run_basic:
    all_roc_data = {}
    detectBodyParts = ['LeftDorsalOrgan', 'RightDorsalOrgan']
    for idx_exp, exp in enumerate(tqdm_notebook(all_exp, unit="exp", desc='Experiments')):
        for idx_test, (test, test_string) in enumerate(tqdm_notebook(test_list, unit="test", desc='Tests')):
            test_dir = os.path.join(root, 'trainingData_'+ exp, test)

            ## Read tracker metadata and annotation
            for fs in os.listdir(test_dir):
                if 'Metadata' in fs:
                    meta_data = pd.read_csv(os.path.join(test_dir, fs), sep=',', header=0, names=meta_data_header)
                if 'Coordinates' in fs:
                    coordinates = pd.read_csv(os.path.join(test_dir, fs), sep=',', names=coordinate_header)

            if (meta_data.empty is False):
                ## Make the metadata frame umber start from 0 index
                meta_data.loc[:, 'FrameNumber'] = meta_data.loc[:, 'FrameNumber'] - 1
                meta_data.set_index('FrameNumber', inplace=True)

                ## Make the annotation frame umber start from 0 index
                coordinates = coordinates.round(0)
                start_frame = coordinates.loc[0, 'FrameNumber'].copy() - 1
                coordinates.loc[:, 'FrameNumber'] = coordinates.loc[:, 'FrameNumber'].copy() - start_frame
                coordinates.set_index('FrameNumber', inplace=True)

                ## Get index of no annotations
                no_anno = np.array(np.where(coordinates.values == -1)).T
                temp_index = coordinates.iloc[no_anno[:, 0], :].index.values
                temp_cols = coordinates.iloc[:,no_anno[:, 1]].columns.values
                no_anno_cols = []
                no_anno_index = []        
                for i in range(0, len(temp_cols), 2):
                    no_anno_cols.append(temp_cols[i][:-2])
                    no_anno_index.append(temp_index[i])
                no_anno_df = pd.DataFrame({'index': no_anno_index, 'col': no_anno_cols})

                ## Inner join to take only the frames intersecting the annotation and tracker metadata
                ## Use when want to see only annotated frames 
                meta_coord = pd.merge(meta_data, coordinates, on='FrameNumber', how='inner', suffixes=('_T', '_A'))

                ## Calculate distance between annotation and the tracker metadata
                for bp in distance_header:
                    temp_x = (meta_coord[bp+'_x_T'].values - meta_coord[bp+'_x_A'].values)**2
                    temp_y = (meta_coord[bp+'_y_T'].values - meta_coord[bp+'_y_A'].values)**2
                    meta_coord.loc[:, bp+'_dist'] = np.sqrt(temp_x + temp_y)*2.75

                ## Replace the no annotation with NA
                for rind, rval in no_anno_df.iterrows():
                    meta_coord.loc[rval['index'], rval['col']+'_dist'] = np.nan

                if idx_test == 0:
                    meta_coord_all_test =  meta_coord
                else:
                    meta_coord_all_test = pd.concat([meta_coord_all_test, meta_coord], axis=0, sort=False)

        all_roc_data[exp] = getROCcurve(meta_coord_all_test, detectBodyParts, outlierDistAll, vote_thresholds)

    rocFile = '20180722_ROC_Data_LDO_RDO.pkl'
    with open(rocFile, 'w') as rf:
        pickle.dump(all_roc_data, rf)

plot_basic = False
if plot_basic:
    detectBodyParts = ['LeftDorsalOrgan', 'RightDorsalOrgan']
    rocFile = '20180722_ROC_Data_LDO_RDO.pkl'
    with open(rocFile, 'r') as rf:
        all_roc_data = pickle.load(rf)
    for exp in all_exp:
        figName = exp + '_ROC_Curve.png'
        plotROC_curve(all_roc_data[exp], detectBodyParts, outlierDistAll, figName)

        figName = exp + '_PR_Curve.png'
        plotPR_curve(all_roc_data[exp], detectBodyParts, outlierDistAll, figName)

        figName = exp + '_Recall_Curve.png'
        plotRecall_curve(all_roc_data[exp], detectBodyParts, outlierDistAll, figName)

run_combi = False
if run_combi:
    all_roc_data = {}
    detectBodyParts = ['And', 'Or']
    for idx_exp, exp in enumerate(tqdm_notebook(all_exp, unit="exp", desc='Experiments')):
        for idx_test, (test, test_string) in enumerate(tqdm_notebook(test_list, unit="test", desc='Tests')):
            test_dir = os.path.join(root, 'trainingData_'+ exp, test)

            ## Read tracker metadata and annotation
            for fs in os.listdir(test_dir):
                if 'Metadata' in fs:
                    meta_data = pd.read_csv(os.path.join(test_dir, fs), sep=',', header=0, names=meta_data_header)
                if 'Coordinates' in fs:
                    coordinates = pd.read_csv(os.path.join(test_dir, fs), sep=',', names=coordinate_header)

            if (meta_data.empty is False):
                ## Make the metadata frame umber start from 0 index
                meta_data.loc[:, 'FrameNumber'] = meta_data.loc[:, 'FrameNumber'] - 1
                meta_data.set_index('FrameNumber', inplace=True)

                ## Make the annotation frame umber start from 0 index
                coordinates = coordinates.round(0)
                start_frame = coordinates.loc[0, 'FrameNumber'].copy() - 1
                coordinates.loc[:, 'FrameNumber'] = coordinates.loc[:, 'FrameNumber'].copy() - start_frame
                coordinates.set_index('FrameNumber', inplace=True)

                ## Get index of no annotations
                no_anno = np.array(np.where(coordinates.values == -1)).T
                temp_index = coordinates.iloc[no_anno[:, 0], :].index.values
                temp_cols = coordinates.iloc[:,no_anno[:, 1]].columns.values
                no_anno_cols = []
                no_anno_index = []        
                for i in range(0, len(temp_cols), 2):
                    no_anno_cols.append(temp_cols[i][:-2])
                    no_anno_index.append(temp_index[i])
                no_anno_df = pd.DataFrame({'index': no_anno_index, 'col': no_anno_cols})

                ## Inner join to take only the frames intersecting the annotation and tracker metadata
                ## Use when want to see only annotated frames 
                meta_coord = pd.merge(meta_data, coordinates, on='FrameNumber', how='inner', suffixes=('_T', '_A'))

                ## Calculate distance between annotation and the tracker metadata
                for bp in distance_header:
                    temp_x = (meta_coord[bp+'_x_T'].values - meta_coord[bp+'_x_A'].values)**2
                    temp_y = (meta_coord[bp+'_y_T'].values - meta_coord[bp+'_y_A'].values)**2
                    meta_coord.loc[:, bp+'_dist'] = np.sqrt(temp_x + temp_y)*2.75

                ## Replace the no annotation with NA
                for rind, rval in no_anno_df.iterrows():
                    meta_coord.loc[rval['index'], rval['col']+'_dist'] = np.nan

                if idx_test == 0:
                    meta_coord_all_test =  meta_coord
                else:
                    meta_coord_all_test = pd.concat([meta_coord_all_test, meta_coord], axis=0, sort=False)

        all_roc_data[exp] = getROCcurve_Combi(meta_coord_all_test, detectBodyParts, outlierDistAll, vote_thresholds)

    rocFile = '20180722_ROC_Data_LDO_RDO_COMBI.pkl'
    with open(rocFile, 'w') as rf:
        pickle.dump(all_roc_data, rf)
        
plot_combi = True
if plot_combi:
    detectBodyParts = ['And', 'Or']
    rocFile = '20180722_ROC_Data_LDO_RDO_COMBI.pkl'
    with open(rocFile, 'r') as rf:
        all_roc_data = pickle.load(rf)
    for exp in all_exp:
        figName = exp + '_ROC_Curve_Combi.png'
        plotROC_curve(all_roc_data[exp], detectBodyParts, outlierDistAll, figName)

        figName = exp + '_PR_Curve_Combi.png'
        plotPR_curve(all_roc_data[exp], detectBodyParts, outlierDistAll, figName)

        figName = exp + '_Recall_Curve_Combi.png'
        plotRecall_curve(all_roc_data[exp], detectBodyParts, outlierDistAll, figName)


0
0
0
1
1
1
0
0
0
1
1
1
0
0
0
1
1
1
0
0
0
1
1
1
